In [1]:
import random

def fuzzer(max_length:int = 100, char_start: int = 32, char_range: int = 32) -> str:
    string_length = random.randrange(0, max_length + 1)
    out = ""
    for i in range(0, string_length):
        out+= chr(random.randrange(char_start, char_start + char_range))
    
    return out


fuzzer()

'08 /5<+>5<>;>=*6&79;8-#576.002,.2;3$+(8,"3-8-2216*-9<\'?,<+."-:& #< "&<""-8:/(.##3.=55+:.\'1,,,'

In [3]:
fuzzer(1000, ord('a'), 26)

'hxpgkrgcmhmwbrnevhnvckxphdwdooijrfxhgzfntalqzcqksohaqrmfpbjtxpqnrmtixkzkniasgwflqgcpdgbjqwfqmqehmuvsdlebtuljjmdorxxtbfaovbzeupbsvxebntmaclnuwsrqgiaoduzrwbghminlpsbhqnzuaxdbtolyhtvdpikgniiuxgcpivfkeoweavqitqrkfmswpwvuibhdcpgajnmmuuhgtncjqhcwznrecwprxiaawkzmeriozocbkhrjthyygybxzedjpyxxnynpjeydegbonrlsqajpiopfjzzfjtxdwqdeevvqrdghknqczhfafgdpvxazcflpjcjzhupjchnkhfdsxbrdmnwubuwcakeprrorzveuluebuacadzsaezvybzxujlnpfkcxzbjgsqpqhkgjodqojzifswozcbdhppxoluugtpyfxpywicltoovtkoehwzfv'

In [4]:
import os
import tempfile

basename = "input.txt"
tempdir = tempfile.mkdtemp()
FILE = os.path.join(tempdir, basename)
print(FILE)

/tmp/tmppwjhch9p/input.txt


In [5]:

data = fuzzer()
with open(FILE, "w") as f:
    f.write(data)

contents = open(FILE).read()
print(contents)
assert(contents == data)

'-!&<:3>+4<4$6=089#!5#:0#0:=


In [6]:
import os
import subprocess

program = "bc"

with open(FILE, "w") as f:
    f.write("2+2\n")

result = subprocess.run([program, FILE], 
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True
                        )

result.stdout

'4\n'

In [9]:
trials = 100
program = "bc"

runs = []

for i in range(trials):
    data = fuzzer()
    with open(FILE, "w") as f:
        f.write(data)
    result = subprocess.run([program, FILE],
                            stdin=subprocess.DEVNULL,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True)
    runs.append((data, result))


sum(1 for (data, result) in runs if result.stderr == "")

5

In [10]:
errors = [(data, result) for (data, result) in runs if result.stderr != ""]

(first_data, first_result) = errors[0]

print(repr(first_data))
print(first_result.stderr)

'7.$:%-"$6"6;785:0?4)?*62?8  />-$78&; '
/tmp/tmppwjhch9p/input.txt 1: illegal character: $
/tmp/tmppwjhch9p/input.txt 1: illegal character: :
/tmp/tmppwjhch9p/input.txt 1: syntax error
/tmp/tmppwjhch9p/input.txt 1: illegal character: :
/tmp/tmppwjhch9p/input.txt 1: syntax error
/tmp/tmppwjhch9p/input.txt 1: illegal character: ?
/tmp/tmppwjhch9p/input.txt 1: illegal character: ?
/tmp/tmppwjhch9p/input.txt 1: illegal character: ?
/tmp/tmppwjhch9p/input.txt 1: illegal character: $



In [11]:
def crash_if_too_long(s):
    buffer = "Thursday"
    if len(s) > len(buffer):
        raise ValueError

from fuzzingbook.ExpectError import ExpectError

trials = 100
with ExpectError():
    for i in range(trials):
        s = fuzzer()
        crash_if_too_long(s)


Traceback (most recent call last):
  File "/tmp/ipykernel_26889/1658604046.py", line 12, in <cell line: 9>
    crash_if_too_long(s)
  File "/tmp/ipykernel_26889/1658604046.py", line 4, in crash_if_too_long
    raise ValueError
ValueError (expected)


In [12]:
with open("program.c", "w") as f:
        f.write("""
#include <stdlib.h>
#include <string.h>

int main(int argc, char** argv) {
    /* Create an array with 100 bytes, initialized with 42 */
    char *buf = malloc(100);
    memset(buf, 42, 100);

    /* Read the N-th element, with N being the first command-line argument */
    int index = atoi(argv[1]);
    char val = buf[index];

    /* Clean up memory so we don't leak */
    free(buf);
    return val;
}
    """)
from fuzzingbook.bookutils import print_file

print_file("program.c")

#include <stdlib.h>
#include <string.h>

int main(int argc, char** argv) {
    /* Create an array with 100 bytes, initialized with 42 */
    char *buf = malloc(100);
    memset(buf, 42, 100);

    /* Read the N-th element, with N being the first command-line argument */
    int index = atoi(argv[1]);
    char val = buf[index];

    /* Clean up memory so we don't leak */
    free(buf);
    return val;
}
    

In [15]:
secrets = ("<space for reply>" + fuzzer(100) +
           "<secret-certificate>" + fuzzer(100) +
           "<secret-key>" + fuzzer(100) + "<other-secrets>")

uninitialized_memory_marker = "deadbeef"
while len(secrets) < 2048:
    secrets += uninitialized_memory_marker


def heartbeat(reply: str, length: int, memory: str) -> str:
    # Store reply in memory
    memory = reply + memory[len(reply):]

    # Send back heartbeat
    s = ""
    for i in range(length):
        s += memory[i]
    return s

In [16]:
heartbeat("potato", 6, memory=secrets)
heartbeat("bird", 4, memory=secrets)

'bird'

In [18]:
heartbeat("hat", 500, memory=secrets)

'hatace for reply>;4$2*1<<,22)+6->4;\'&(95(,23/"=$90;/,-\',8-+!2<secret-certificate>8"%+(;&#9;3)),?2%-#>=*6=?=.#?9> 1*;+"\'&6: 499!?#3#&5;?-\'**(8:>)/< >00&<4$?<:#*?!(=-2&9-4?\'4)1$(4<<secret-key>4-(>!9\'%/0<),;;.=&*/(")"#:.<-&*5;<other-secrets>deadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbe'

In [19]:
from fuzzingbook.ExpectError import ExpectError

with ExpectError():
    for i in range(10):
        s = heartbeat(fuzzer(), random.randint(1,500), memory=secrets)
        assert not s.find(uninitialized_memory_marker)
        assert not s.find("secret")

Traceback (most recent call last):
  File "/tmp/ipykernel_26889/216920107.py", line 6, in <cell line: 3>
    assert not s.find(uninitialized_memory_marker)
AssertionError (expected)


In [21]:


from typing import Dict


airport_codes: Dict[str, str] = {
    "YVR": "Vancouver",
    "JFK": "New York-JFK",
    "CDG": "Paris-Charles de Gaulle",
    "CAI": "Cairo",
    "LED": "St. Petersburg",
    "PEK": "Beijing",
    "HND": "Tokyo-Haneda",
    "AKL": "Auckland"
}  # plus many more

airport_codes["YVR"]

'Vancouver'

In [22]:
def code_repOK(code:str) -> bool :
    # repr 返回对象的string格式
    assert len(code) == 3, "Airport code must have three characters: " + repr(code) 
    for c in code:
        # 检查一个字符串中所有的字符是否都是由字母构成的，并且至少有1个字符。
        assert c.isalpha(), "Non-letter in airport code: " + repr(code)
        # 用于判断字符串里的字母是否都是大写，如果某个字符本身并不区分大小写，那么不参与判断，如果字符串里没有大写字母，会返回False
        assert c.isupper(), "Lowercase letter in airport code: " + repr(code)
    return True

assert code_repOK("SEA")

In [23]:
def airport_codes_repOK():
    for code in airport_codes:
        assert code_repOK(code)
    return True

with ExpectError():
    assert airport_codes_repOK()

airport_codes["YMML"] = "Melbourne"

with ExpectError():
    assert airport_codes_repOK()

Traceback (most recent call last):
  File "/tmp/ipykernel_26889/634072318.py", line 12, in <cell line: 11>
    assert airport_codes_repOK()
  File "/tmp/ipykernel_26889/634072318.py", line 3, in airport_codes_repOK
    assert code_repOK(code)
  File "/tmp/ipykernel_26889/1554571450.py", line 3, in code_repOK
    assert len(code) == 3, "Airport code must have three characters: " + repr(code)
AssertionError: Airport code must have three characters: 'YMML' (expected)


In [24]:
def add_new_airport(code: str, city: str) -> None:
    assert code_repOK(code)
    airport_codes[code] = city

with ExpectError():  # For BER, ExpectTimeout would be more appropriate
    add_new_airport("BER", "Berlin")

with ExpectError():
    add_new_airport("London-Heathrow", "LHR")

Traceback (most recent call last):
  File "/tmp/ipykernel_26889/3928306651.py", line 9, in <cell line: 8>
    add_new_airport("London-Heathrow", "LHR")
  File "/tmp/ipykernel_26889/3928306651.py", line 2, in add_new_airport
    assert code_repOK(code)
  File "/tmp/ipykernel_26889/1554571450.py", line 3, in code_repOK
    assert len(code) == 3, "Airport code must have three characters: " + repr(code)
AssertionError: Airport code must have three characters: 'London-Heathrow' (expected)


In [25]:
def add_new_airport_2(code: str, city: str) -> None:
    assert code_repOK(code)
    assert airport_codes_repOK()
    airport_codes[code] = city
    assert airport_codes_repOK()

with ExpectError():
    add_new_airport_2("IST", "Istanbul Yeni Havalimanı")

Traceback (most recent call last):
  File "/tmp/ipykernel_26889/3975422841.py", line 8, in <cell line: 7>
    add_new_airport_2("IST", "Istanbul Yeni Havalimanı")
  File "/tmp/ipykernel_26889/3975422841.py", line 3, in add_new_airport_2
    assert airport_codes_repOK()
  File "/tmp/ipykernel_26889/634072318.py", line 3, in airport_codes_repOK
    assert code_repOK(code)
  File "/tmp/ipykernel_26889/1554571450.py", line 3, in code_repOK
    assert len(code) == 3, "Airport code must have three characters: " + repr(code)
AssertionError: Airport code must have three characters: 'YMML' (expected)
